In [1]:
!pip install nbformat
import torchvision
import numpy as np
import matplotlib
import wandb
from src import preprocess, activation_functions, initializers, layer, loss_functions, optimizers, FFNN, training, wandblogger, dropout, batchnorm
import nbformat


   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   --------------- ------------------------ 30.7/78.5 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 78.5/78.5 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/90.0 kB ? eta -:--:--
   ---------------------------------------- 90.0/90.0 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/67.6 kB ? eta -:--:--
   ---------------------------------------- 67.6/67.6 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/235.7 kB ? eta -:--:--
   ---------------------------------------- 235.7/235.7 kB 4.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=False)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=False)

In [3]:
validation_percent = 0.2
one_hot_encoder = preprocess.OneHotEncoder()


train_size = int((1 - validation_percent) * len(train_dataset))
validation_size = len(train_dataset) - train_size

x_train = train_dataset.data[:train_size].reshape(train_size, -1)
targets_train = np.array(train_dataset.targets[:train_size])
y_train = one_hot_encoder.transform(targets_train)

x_valid = train_dataset.data[train_size:].reshape(validation_size, -1)
targets_valid = np.array(train_dataset.targets[train_size:])
y_valid = one_hot_encoder.transform(targets_valid)

x_test = test_dataset.data.reshape(len(test_dataset), -1)
targets_test = np.array(test_dataset.targets)
y_test = one_hot_encoder.transform(targets_test)

print(type(x_train), type(y_train))
print(x_train.shape, y_train.shape)

<class 'torch.Tensor'> <class 'numpy.ndarray'>
torch.Size([48000, 784]) (48000, 10)


C:\Users\Ludvig\AppData\Local\Temp\ipykernel_33816\3457253408.py:9: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  targets_train = np.array(train_dataset.targets[:train_size])
C:\Users\Ludvig\AppData\Local\Temp\ipykernel_33816\3457253408.py:13: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  targets_valid = np.array(train_dataset.targets[train_size:])
C:\Users\Ludvig\AppData\Local\Temp\ipykernel_33816\3457253408.py:17: DeprecationWarning: __array__ implementation doesn't accep

In [4]:
#find min and max value of y_train
print("y_train min:", np.min(y_train), "y_train max:", np.max(y_train))

y_train min: 0.0 y_train max: 1.0


In [ ]:
project = "Fashion-MNIST"
entity = "DTU-Deep-Learning-Project"

In [ ]:
def train_sweep():
    wandb.init(entity=entity,
        project=project)        
    cfg = wandb.config  
    
    # building layers dynamically
    layers = []

    input_size = 28 * 28
    prev_size = input_size
    # hidden layers
    for hidden_size in cfg.layer_sizes:     
        layers.append(
            layer.Layer(
                input_size=prev_size,
                output_size=hidden_size,
                weight_initializer=initializers.HeInitializer(),
                bias_initializer=initializers.ConstantInitializer(0.0),
            )
        )
        # add batchnorm if specified
        if cfg.use_batchnorm:
            layers.append(batchnorm.BatchNorm(hidden_size))

        layers.append(activation_functions.ReLU())
        # add dropout if specified, had issues with 0.5 dropout one round
        if cfg.dropout_rate > 0:
            layers.append(dropout.Dropout(cfg.dropout_rate))

        prev_size = hidden_size

    # Output layer
    layers.append(
        layer.Layer(
            input_size=prev_size,
            output_size=10,
            weight_initializer=initializers.HeInitializer(),
            bias_initializer=initializers.ConstantInitializer(0.0),
        )
    )

    if cfg.optimizer == "sgd":
        optimizer = optimizers.SGD(learning_rate=cfg.learning_rate)
    elif cfg.optimizer == "adam":
        optimizer = optimizers.Adam(learning_rate=cfg.learning_rate)
    
    model = FFNN.FFNN(
        layers=layers,
        loss_function=loss_functions.CrossEntropyLoss(),
        optimizer=optimizer,
    )

    logger = wandblogger.Logger(project)
    trainer = training.Trainer(model=model, loss_function=loss_functions.CrossEntropyLoss(), optimizer=optimizer, logger=logger)

    history = trainer.train(
        x_train,
        y_train,x_val=x_valid,
        y_val=y_valid,
        epochs=cfg.epochs,
        batch_size=cfg.batch_size,
        shuffle=True
    )

    wandb.finish()


In [ ]:
sweep_config = {
    "method": "random",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "parameters": {
        "optimizer": {"values": ["sgd"]},
        "learning_rate": {"min": 0.0005, "max": 0.1},
        "batch_size": {"values": [32, 64, 128]},
        "epochs": {"value": 20},

        # 🔥 The important part:
        # Hidden layer configurations can vary
        "layer_sizes": {
            "values": [
                [128],
                [128, 64],
                [256, 128, 64],
                [64, 64, 64, 64]
            ]
        },
        
        "use_batchnorm": {"values": [True, False]},
        "dropout_rate": {"values": [0.0, 0.3, 0.5]},
    },
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity=entity,
    project=project)

wandb.agent(sweep_id, function=train_sweep, count=10)

Create sweep with ID: 2cxwnzbq
Sweep URL: https://wandb.ai/DTU-Deep-Learning-Project/Fashion-MNIST/sweeps/2cxwnzbq


wandb: Agent Starting Run: x4hmr0uf with config:
wandb: 	batch_size: 64
wandb: 	dropout_rate: 0.5
wandb: 	epochs: 20
wandb: 	layer_sizes: [128]
wandb: 	learning_rate: 0.08183196155283931
wandb: 	optimizer: sgd
wandb: 	use_batchnorm: True
wandb: Currently logged in as: s254344 (DTU-Deep-Learning-Project) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Traceback (most recent call last):
  File "c:\Users\Ludvig\OneDrive - Danmarks Tekniske Universitet\Kandidat\Deep learning\Deep-Learning-Project\.venv\Lib\site-packages\wandb\agents\pyagent.py", line 297, in _run_job
    self._function()
  File "C:\Users\Ludvig\AppData\Local\Temp\ipykernel_33816\2709711375.py", line 56, in train_sweep
    history = trainer.train(
              ^^^^^^^^^^^^^^
TypeError: Trainer.train() missing 1 required positional argument: 'early_stopper'



wandb: ERROR Run x4hmr0uf errored: Trainer.train() missing 1 required positional argument: 'early_stopper'
wandb: Agent Starting Run: e4plgjus with config:
wandb: 	batch_size: 32
wandb: 	dropout_rate: 0.3
wandb: 	epochs: 20
wandb: 	layer_sizes: [128, 64]
wandb: 	learning_rate: 0.08060428725224474
wandb: 	optimizer: sgd
wandb: 	use_batchnorm: False


Traceback (most recent call last):
  File "c:\Users\Ludvig\OneDrive - Danmarks Tekniske Universitet\Kandidat\Deep learning\Deep-Learning-Project\.venv\Lib\site-packages\wandb\agents\pyagent.py", line 297, in _run_job
    self._function()
  File "C:\Users\Ludvig\AppData\Local\Temp\ipykernel_33816\2709711375.py", line 56, in train_sweep
    history = trainer.train(
              ^^^^^^^^^^^^^^
TypeError: Trainer.train() missing 1 required positional argument: 'early_stopper'



wandb: ERROR Run e4plgjus errored: Trainer.train() missing 1 required positional argument: 'early_stopper'
wandb: Agent Starting Run: auk15rjz with config:
wandb: 	batch_size: 128
wandb: 	dropout_rate: 0.3
wandb: 	epochs: 20
wandb: 	layer_sizes: [64, 64, 64, 64]
wandb: 	learning_rate: 0.05171950691243535
wandb: 	optimizer: sgd
wandb: 	use_batchnorm: True


Traceback (most recent call last):
  File "c:\Users\Ludvig\OneDrive - Danmarks Tekniske Universitet\Kandidat\Deep learning\Deep-Learning-Project\.venv\Lib\site-packages\wandb\agents\pyagent.py", line 297, in _run_job
    self._function()
  File "C:\Users\Ludvig\AppData\Local\Temp\ipykernel_33816\2709711375.py", line 56, in train_sweep
    history = trainer.train(
              ^^^^^^^^^^^^^^
TypeError: Trainer.train() missing 1 required positional argument: 'early_stopper'



wandb: ERROR Run auk15rjz errored: Trainer.train() missing 1 required positional argument: 'early_stopper'
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true
